# Importintg

In [1]:
import pyspark 
import matplotlib.pyplot as plt
import findspark
import pandas as pd
from pyspark.sql.types import *
from pyspark.sql.functions import *

# Spark setup Check

In [2]:
import findspark

findspark.init()



In [3]:

from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

# df = spark.sql("select 'spark' as hello ")

# df.show()


## Data Read

In [4]:

df= spark.read.load(
    "prodapt/", 
    format="csv", sep=",", inferSchema="true", header="true"
)

df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- location_id: string (nullable = true)
 |-- client_mac: string (nullable = true)
 |-- freq_band: string (nullable = true)
 |-- mac_address: string (nullable = true)
 |-- country: string (nullable = true)
 |-- vendor_name: string (nullable = true)
 |-- rssi_percs_25: double (nullable = true)
 |-- rssi_percs_75: double (nullable = true)
 |-- isp: string (nullable = true)
 |-- total_bytes_tx: double (nullable = true)
 |-- total_bytes_rx: double (nullable = true)
 |-- ts: string (nullable = true)
 |-- txWeightedPhyRate: double (nullable = true)
 |-- rxWeightedPhyRate: double (nullable = true)
 |-- idtype: string (nullable = true)
 |-- is_static: boolean (nullable = true)
 |-- weighted_prr_tx: double (nullable = true)
 |-- weighted_prr_rx: double (nullable = true)
 |-- num_samples: integer (nullable = true)
 |-- is_connected_to_pod: boolean (nullable = true)
 |-- pod_serial: string (nullable = true)
 |-- control_group: string (nullable = true)
 |

## Frequency Band percentage

In [8]:
tot = df.count()
df.groupBy("freq_band") \
  .count() \
  .withColumnRenamed('count', 'countFreqBand') \
  .withColumn('perc_countFreqBand', (col('countFreqBand') / tot) * 100 ) \
  .show()

+---------+-------------+------------------+
|freq_band|countFreqBand|perc_countFreqBand|
+---------+-------------+------------------+
|     2.4G|       314735| 44.92389337078716|
|       5G|       385861| 55.07610662921284|
+---------+-------------+------------------+



Now model the a future hypothetical scenario that could be caused after the new Band Steering feature is activated. In this future scenario the
following four conditions will be met:
• Every Device connected to 2.4 GHz with an RSSI >= -60 dB connects to 5 GHz
• Every Device connected to 5 GHz with an RSSI <= -75 dB connects to 2.4 GHz
• Every Device connected to 5 GHz with an RSSI > -75 dB stays on 5GHz
• Every Device connected to 2.4 GHz with an RSSI < -60 dB stays on 2.4GHz


## Band steering scenario

In [28]:
df = df.withColumn(
    'NEWfreq_band',
    when((col("freq_band") == '2.4G') & (col("rssi_percs_25") >=-60), '5G')\
    .when((col("freq_band") == '2.4G') & (col("rssi_percs_25") < -60), '2.4G')\
    .when((col("freq_band") == '5G') & (col('rssi_percs_25') >-75), '5G')\
    .when((col("freq_band") == '5G') & (col('rssi_percs_25') <= -75), '2.4G')\
    .otherwise(col("freq_band"))

)


In [19]:
df.summary()

DataFrame[summary: string, _c0: string, location_id: string, client_mac: string, freq_band: string, mac_address: string, country: string, vendor_name: string, rssi_percs_25: string, rssi_percs_75: string, isp: string, total_bytes_tx: string, total_bytes_rx: string, ts: string, txWeightedPhyRate: string, rxWeightedPhyRate: string, idtype: string, weighted_prr_tx: string, weighted_prr_rx: string, num_samples: string, pod_serial: string, control_group: string, maxMcs: string, maxNss: string, maxChWidth: string, phyMode: string, NEWfreq_band: string]

## New Frequency percentage
after putting into effect the above conditions, we were able to increase 5G users by 15%

In [29]:
tot = df.count()
df.groupBy("NEWfreq_band") \
  .count() \
  .withColumnRenamed('count', 'countFreqBand') \
  .withColumn('perc_countFreqBand', (col('countFreqBand') / tot) * 100 ) \
  .show()

+------------+-------------+------------------+
|NEWfreq_band|countFreqBand|perc_countFreqBand|
+------------+-------------+------------------+
|        2.4G|       208277|29.728545409908136|
|          5G|       492319| 70.27145459009186|
+------------+-------------+------------------+

